In [12]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from apikey import apikey 
import os
os.environ['OPENAI_API_KEY'] = apikey


# Quickstart Guide

In [ ]:
# 关于prompt 的使用方法
# 官网链接：https://python.langchain.com/en/latest/getting_started/getting_started.html#chat-prompt-templates

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages())
# -> AIMessage(content="J'aime programmer.", additional_kwargs={})

In [ ]:
# 结合了chain 之后的用法，这种写法就更方便了
# 官网链接：https://python.langchain.com/en/latest/getting_started/getting_started.html#chains-with-chat-models

from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="French", text="I love programming.")
# -> "J'aime programmer."

In [ ]:
# 留个作业，看下以下返回有什么不同？
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

chat = ChatOpenAI(temperature=0)
llm = OpenAI(temperature=0)

In [ ]:
# ReAct ：is a prompting technique that combines Chain-of-Thought prompting with action plan generation. 
# This induces the model to think about what action to take, then take it.
# 这里加深下对Agents的理解
# 官方网站：https://python.langchain.com/en/latest/modules/agents/agents/examples/react.html
# 其实Agents不只是做查询，他是一系列的工具，比如这里
# 工具就比较实用；

from langchain import OpenAI, Wikipedia
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.agents.react.base import DocstoreExplorer
docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search,
        description="useful for when you need to ask with search"
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup,
        description="useful for when you need to ask with lookup"
    )
]

llm = OpenAI(temperature=0, model_name="text-davinci-002")
react = initialize_agent(tools, llm, agent=AgentType.REACT_DOCSTORE, verbose=True)

question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
react.run(question)



In [ ]:
# Self-ask：is a prompting method that builds on top of chain-of-thought prompting. 
# In this method, the model explicitly asks itself follow-up questions, which are then answered by an external search engine.

from langchain import OpenAI, SerpAPIWrapper
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search"
    )
]

self_ask_with_search = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)
self_ask_with_search.run("What is the hometown of the reigning men's U.S. Open champion?")

# Models

In [8]:
from langchain.llms import OpenAI
from langchain.llms.loading import load_llm


In [10]:
!cat llm.json


{
    "model_name": "text-davinci-003",
    "temperature": 0.7,
    "max_tokens": 256,
    "top_p": 1.0,
    "frequency_penalty": 0.0,
    "presence_penalty": 0.0,
    "n": 1,
    "best_of": 1,
    "request_timeout": null,
    "_type": "openai"
}

In [13]:
# 这种用法应该会用的到，将 llm的参数作为配置文件，然后直接用 load_llm 加载即可。
# 见官网：https://python.langchain.com/en/latest/modules/models/llms/examples/llm_serialization.html

llm = load_llm("llm.json")
llm

OpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.7, max_tokens=256, top_p=1.0, frequency_penalty=0.0, presence_penalty=0.0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False, allowed_special=set(), disallowed_special='all')

In [ ]:
# 对于token的track 也可能会用到
# 见官网：https://python.langchain.com/en/latest/modules/models/llms/examples/token_usage_tracking.html

In [14]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate
string_prompt = PromptTemplate.from_template("tell me a joke about {subject}")
chat_prompt = ChatPromptTemplate.from_template("tell me a joke about {subject}")
string_prompt_value = string_prompt.format_prompt(subject="soccer")
chat_prompt_value = chat_prompt.format_prompt(subject="soccer")

In [19]:
string_prompt_value.to_messages()

[HumanMessage(content='tell me a joke about soccer', additional_kwargs={}, example=False)]

In [20]:
chat_prompt_value.to_messages()

[HumanMessage(content='tell me a joke about soccer', additional_kwargs={}, example=False)]

In [21]:
# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a joke.")
no_input_prompt

PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Tell me a joke.', template_format='f-string', validate_template=True)

In [24]:
no_input_prompt.format()


'Tell me a joke.'

In [ ]:
template = "I am learning langchain because {reason}."

prompt_template = PromptTemplate(template=template, 
                                 input_variables=["reason", "foo"]) # ValueError due to extra variables

prompt_template = PromptTemplate(template=template, 
                                 input_variables=["reason", "foo"], 
                                 validate_template=False) # No error



## FewShotPromptTemplate

这块儿要研究一下，后面考虑可以对输出的json做一个例子；

In [25]:
from langchain import PromptTemplate, FewShotPromptTemplate

# First, create the list of few shot examples.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
]

# Next, we specify the template to format the examples we have provided.
# We use the `PromptTemplate` class for this.
example_formatter_template = """Word: {word}
Antonym: {antonym}
"""
example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=example_formatter_template,
)

example_prompt

PromptTemplate(input_variables=['word', 'antonym'], output_parser=None, partial_variables={}, template='Word: {word}\nAntonym: {antonym}\n', template_format='f-string', validate_template=True)

In [26]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    examples=examples,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="Give the antonym of every input\n",
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="Word: {input}\nAntonym: ",
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [27]:
# We can now generate a prompt using the `format` method.
print(few_shot_prompt.format(input="big"))


Give the antonym of every input

Word: happy
Antonym: sad

Word: tall
Antonym: short

Word: big
Antonym: 


## LengthBasedExampleSelector

In [28]:
from langchain.prompts.example_selector import LengthBasedExampleSelector

# These are a lot of examples of a pretend task of creating antonyms.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "energetic", "antonym": "lethargic"},
    {"word": "sunny", "antonym": "gloomy"},
    {"word": "windy", "antonym": "calm"},
]

# We'll use the `LengthBasedExampleSelector` to select the examples.
example_selector = LengthBasedExampleSelector(
    # These are the examples is has available to choose from.
    examples=examples, 
    # This is the PromptTemplate being used to format the examples.
    example_prompt=example_prompt, 
    # This is the maximum length that the formatted examples should be.
    # Length is measured by the get_text_length function below.
    max_length=25
    # This is the function used to get the length of a string, which is used
    # to determine which examples to include. It is commented out because
    # it is provided as a default value if none is specified.
    # get_text_length: Callable[[str], int] = lambda x: len(re.split("\n| ", x))
)

# We can now use the `example_selector` to create a `FewShotPromptTemplate`.
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Word: {input}\nAntonym:",
    input_variables=["input"],
    example_separator="\n\n",
)

# We can now generate a prompt using the `format` method.
print(dynamic_prompt.format(input="big"))

Give the antonym of every input

Word: happy
Antonym: sad


Word: tall
Antonym: short


Word: energetic
Antonym: lethargic


Word: sunny
Antonym: gloomy


Word: big
Antonym:


In [29]:
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(input=long_string))

Give the antonym of every input

Word: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Antonym:


In [ ]:
from feast import FeatureStore

# You may need to update the path depending on where you stored it
feast_repo_path = "../../../../../my_feature_repo/feature_repo/"
store = FeatureStore(repo_path=feast_repo_path)

In [31]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [32]:
template = """Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: {conv_rate}
Acceptance rate: {acc_rate}
Average Daily Trips: {avg_daily_trips}

Your response:"""

prompt = PromptTemplate.from_template(template)
prompt

PromptTemplate(input_variables=['acc_rate', 'avg_daily_trips', 'conv_rate'], output_parser=None, partial_variables={}, template="Given the driver's up to date stats, write them note relaying those stats to them.\nIf they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better\n\nHere are the drivers stats:\nConversation rate: {conv_rate}\nAcceptance rate: {acc_rate}\nAverage Daily Trips: {avg_daily_trips}\n\nYour response:", template_format='f-string', validate_template=True)

In [33]:
class FeastPromptTemplate(StringPromptTemplate):
    
    def format(self, **kwargs) -> str:
        driver_id = kwargs.pop("driver_id")
        feature_vector = store.get_online_features(
            features=[
                'driver_hourly_stats:conv_rate',
                'driver_hourly_stats:acc_rate',
                'driver_hourly_stats:avg_daily_trips'
            ],
            entity_rows=[{"driver_id": driver_id}]
        ).to_dict()
        kwargs["conv_rate"] = feature_vector["conv_rate"][0]
        kwargs["acc_rate"] = feature_vector["acc_rate"][0]
        kwargs["avg_daily_trips"] = feature_vector["avg_daily_trips"][0]
        return prompt.format(**kwargs)
    
prompt_template = FeastPromptTemplate(input_variables=["driver_id"])



In [34]:
print(prompt_template.format(driver_id=1001))


NameError: name 'store' is not defined